In [3]:
# import all necessary packeges
import pandas as pd
import xml.etree.ElementTree as ET, getopt, logging, sys, random, re, copy
from xml.sax.saxutils import escape
import io


from PIL import Image
import requests
from io import BytesIO
#import matplotlib.pyplot as plt
import numpy as np

import warnings

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import math
import time
import re
import os
#import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
#from matplotlib import gridspec
from scipy.sparse import hstack
#import plotly4
#import plotly.figure_factory as ff
#from plotly.graph_objs import Scatter, Layout

#plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
print("done")

#
#
# Parsing from XML File
#
#

xml_data1 = open('IN_OUT/Hin_ABSA_ATE.xml', encoding="utf8").read()
#xml_data = io.StringIO(xml_data1)

def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = [] #This is our record list which we will convert into a dataframe
    for i, child in enumerate(root): #Begin looping through our root tree
        record = {} #Place holder for our record
        for subchild in child: #iterate through the subchildren to user-agent, Ex: ID, String, Description.
            if(subchild.tag == 'text'):
                record[subchild.tag] = subchild.text
                all_records.append(record)
            
    return pd.DataFrame(all_records) #return records as DataFrame


#  Fetching Id

def iter_docs(sentences):
    author_attr = sentences.attrib
    for doc in sentences.iter('sentence'):
        doc_dict = author_attr.copy()
        doc_dict.update(doc.attrib)  # doc.attribute give the value of the sentence id e.g. {'id': 12334}
        yield doc_dict

xml_data = io.StringIO(xml_data1)
etree = ET.parse(xml_data) #create an ElementTree object 
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))
sentence_data = xml2df(xml_data1)
print(doc_df)
print(sentence_data)

#
#
# Storing in pandas dataframe aspect_terms
#
#

# To record the set the set of aspet terms in the review sentences
aspects_records = []
for node in etree.iter(): 
    
    record = {}
    
    if(node.tag == 'sentence'):
        id = node.attrib.get('id')
    
    if (node.tag == 'text'):
        text =  node.text      
    
    if (node.tag == 'aspectTerm' ):
        record['id'] = id 
        record['text'] = text 
        record['term'] = node.attrib.get('term') 
        record['polarity'] = node.attrib.get('polarity') 
        record['from'] = node.attrib.get('from') 
        record['to'] = node.attrib.get('to') 
        aspects_records.append(record) 
    
      

aspect_terms = pd.DataFrame(aspects_records)

#
#
# Writing dataframe to pickel file
#
#
aspect_terms.to_pickle('pickels/180k_aspect_terms')




done
           id
0       app_2
1       app_3
2       app_4
3       app_5
4       app_6
5       app_7
6       app_8
7       app_9
8      app_10
9      app_11
10     app_12
11     app_13
12     app_14
13     app_15
14     app_16
15     app_17
16     app_18
17     app_19
18     app_20
19     app_21
20     app_22
21     app_23
22     app_24
23     app_25
24     app_26
25     app_27
26     app_28
27     app_29
28     app_30
29     app_31
...       ...
5387  tel_106
5388  tel_107
5389  tel_108
5390  tel_109
5391  tel_110
5392  tel_111
5393  tel_112
5394  tel_113
5395  tel_114
5396  tel_115
5397  tel_116
5398  tel_117
5399  tel_118
5400  tel_119
5401  tel_120
5402  tel_121
5403  tel_122
5404  tel_123
5405  tel_124
5406  tel_125
5407  tel_126
5408  tel_127
5409  tel_128
5410  tel_129
5411  tel_130
5412  tel_132
5413  tel_133
5414  tel_134
5415  tel_135
5416  tel_136

[5417 rows x 1 columns]
                                                   text
0     फेसबुक का सिक्योरिटी चैकअप फीचर पॉपअप की

FileNotFoundError: [Errno 2] No such file or directory: 'pickels/180k_aspect_terms'

In [3]:
#
#
# Converting Parsed output into a pandas dataframe word_pos_df
# by storing
# Word, position, line_number, Part of speech 

import pandas as pd
with open('IN_OUT/file1_final.txt',encoding='utf8') as f:
    
    line = f.readline()
    line_no = 0
    word_pos = []
   
        
    while line:
        # print("Line {}: {}".format(cnt, line.strip()))
        
        s1 = line.split(' ') 
        s1 = s1[:-1] 
        #print (s1) 
        word_no = 0
        record = {}
        for i in range (0, len(s1), 3):
            record = {}
            record['word'] =  (s1[i][:-2])
            record['pos'] = (s1[i+1])
            record['word_no'] = word_no
            record['line_no'] = line_no
            word_no = word_no + 1
            word_pos.append(record)   
        
        line = f.readline()
        line_no += 1
    
        
f.close()
word_pos_df = pd.DataFrame(word_pos)
print('done')

done


In [4]:
word_pos_df

,line_no,pos,word,word_no
0,0,NN,फेसबुक,0
1,0,PSP,का,1
2,0,NNP,सिक्योरिटी,2
3,0,NNP,चैकअप,3
4,0,NNP,फीचर,4
5,0,NNP,पॉपअप,5
6,0,PSP,की,6
7,0,PSP,तरह,7
8,0,NNP,यूजर्स,8
9,0,PSP,को,9


In [25]:
# Building a training data & testing data for each domain 
# using
# aspect_terms dataframe and POS-file: file1_final.txt
# in
# 12 TRAINING_domain and TESTING_domain files
#
#
# Partitioning as 

# Create a table of each word in the sentence where 1st column is sentence_id, 2nd is word , 3rd from word is started and
# 4 th BIO (if B of boundry, Inside , Outside)
table2 = []

suffixes = {
    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],
    2: ["कर", "ाओ", "िए", "ाई", "ाए", "ने", "नी", "ना", "ते", "ीं", "ती", "ता", "ाँ", "ां", "ों", "ें"],
    3: ["ाकर", "ाइए", "ाईं", "ाया", "ेगी", "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं"],
    4: ["ाएगी", "ाएगा", "ाओगी", "ाओगे", "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां"],
    5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां"],
}

def get_suffix(word):
    suf='none'    
    for L in 5, 4, 3, 2, 1:
        if len(word) > L + 1:
            for suf in suffixes[L]:
                if word.endswith(suf):
                    return(suf)
    return(suf)
    

b = get_suffix("फेसबुक")
print(b)

def get_label(sentence_id, word, loc):
    label='O'
    cnt_asp = aspect_terms.loc[(aspect_terms['id'] == sentence_id), 'from'].count()
    for i in range(0, cnt_asp):
        term_from = int(aspect_terms.loc[(aspect_terms['id'] == sentence_id) , 'from'].iloc[i]) # int(row['from'])
        term_upto = int(aspect_terms.loc[(aspect_terms['id'] == sentence_id) , 'to'].iloc[i]) #int(row['to'])
        if loc==term_from:
            label='B'
            return(label)
        elif loc>term_from and loc<term_upto:
            label='I'
            return(label)
    if cnt_asp == 0:
        #print ("Sentence Index Error")
        label = "O"
        
    return(label)

def get_pos(line_num, wno):
    return(word_pos_df.loc[(word_pos_df['line_no'] == line_num) & (word_pos_df['word_no'] == wno), 'pos'].iloc[0])



def get_data(loc_from, loc_to, file_name, trte):
    
    num = loc_from;
    train_file_name = "IN_OUT/" + trte + "_" + file_name + ".txt"
    #out = open("IN_OUT/LAP_TRAIN.txt", "w")
    #print('TFN', train_file_name)
    train = open(train_file_name, "a+", encoding="utf8")
    #test = open("IN_OUT/TEST.txt", "w")

    while num < loc_to: # 749:
        
        print(num) 
        sentence_text = sentence_data['text'][num]
        sentence_id = doc_df['id'][num]
    
        loc = 0
        word_num = 0
        for word in sentence_text.split():
            label = get_label(sentence_id, word, loc)
            record={}
            # record['id'] = sentence_id
            record['aword'] = word
            # record['w_from'] = loc       
            record['dBIO'] = label
            suf1 = get_suffix(word)
            record['cw_suff'] = suf1
            pos1 = get_pos(num, word_num)
            record['bpos'] = pos1   # num means line_num
            table2.append(record)
            loc += len(word) + 1
            word_num += 1
            train.write(word + " " + pos1 + " " + suf1 + " " + label + "\n")
            #test.write(word + "\t" + pos1 + "\t" + suf1 + "\t" + label + "\n")
        
        num = num + 1
        train.write("\n")
    table2_pd = pd.DataFrame(table2)
    print("done")
    train.close()
    table2_pd 
    return("done")

#y = get_data(507, 507+243, "Laptop","TRAINING" )
#y = get_data(507+244,507+244+105, "Laptop","TESTING" )

print("we start")
# RUN 1
# start = [0,230,379,422,506,854,1995,2885,3215,3262,4506,5282]
# training_count = [160,105,29,57,243,799,623,231,33,871,543,95]
# file_names = ['Mobile_apps','Camera','Headphones','Home_appliances','Laptop','Mobile','Movies','Smart_watch','Speakers','Tablets','Travels','Televisions']

start = [0,230,379,422,506,854,1995,2885,3215,3262,4506,5282]

total_count = [229,150,43,84,348,1141,890,330,47,1244,776,135]
training_count = [160,105,29,57,243,799,623,231,33,871,543,95]

#testing_count = 
def get_train_test_data(num):
    print(num) 
    table2 = []
    sentence_text = sentence_data['text'][num]
    sentence_id = doc_df['id'][num]

    loc = 0
    word_num = 0
    for word in sentence_text.split():
        label = get_label(sentence_id, word, loc)
        record={}
        # record['id'] = sentence_id
        record['word'] = word
        # record['w_from'] = loc       
        record['BIO'] = label
        suf1 = get_suffix(word)
        record['w_suff'] = suf1
        pos1 = get_pos(num, word_num)
        record['pos'] = pos1   # num means line_num
        table2.append(record)
        loc += len(word) + 1
        word_num += 1
        #train.write(word + " " + pos1 + " " + suf1 + " " + label + "\n")
        
    return table2 


x = (get_train_test_data(0))
print(len(x))



ा
we start
0
12


In [ ]:
start = [0,230,379,422,506,854,1995,2885,3215,3262,4506,5282]

total_count = [229,150,43,84,348,1141,890,330,47,1244,776,135]
training_count = [160,105,29,57,243,799,623,231,33,871,543,95]

testing_count = 

In [7]:
# Aspect Term detection 
# confusion Matrix Intialization
# 
# RUN 1: 
####################################


detected_list = ['ATD', 'NATD']
actual_list = ['ATO', 'NATO']
BIO_CM = np.zeros((2, 2))

#names = [_ for _ in confusion_mat_words_list ]
BIO_CM = pd.DataFrame(BIO_CM, index=actual_list, columns=detected_list)
BIO_CM

file_names = ['Mobile_apps'] #,'Camera','Headphones','Home_appliances','Laptop','Mobile','Movies','Smart_watch','Speakers','Tablets','Travels','Televisions','ALL']
file_no = 0
confusion_matrix_of_file = "IN_OUT/" + 'Output' + "_ALL.txt" # + file_names[file_no] + ".txt"

ALL_OUTPUTS = []
Output = {}
temp_out = open("IN_OUT\Run123.txt", "w")
for file_no in range(0,len(file_names)):
    
    #confusion_matrix_of_file = "IN_OUT/" + 'Output' + "_" + file_names[file_no] + ".txt"
    BIO_CM = np.zeros((2, 2))
    
    BIO_CM = pd.DataFrame(BIO_CM, index=actual_list, columns=detected_list)
    
    with open(confusion_matrix_of_file, encoding='utf8') as f:
    
        line = f.readline()
        line_no = 0 
        
        while line:
            # print("Line {}: {}".format(cnt, line.strip()))
        
            s1 = line.split('\t') 
            print (s1)
            if len(s1) > 1:
                print(s1[-2])
                print (s1[-1][0])
            
                if s1[-2] == 'O' and s1[-1][0] =='O' :          # CASE I 'OO'
                    BIO_CM.loc['NATO', 'NATD'] = BIO_CM.loc['NATO', 'NATD'] + 1 
                    line = f.readline()
                    continue
                elif s1[-2] == 'B' and s1[-1][0] =='B' :         # CASE II 'BB'
                    BIO_CM.loc['ATO', 'ATD'] = BIO_CM.loc['ATO', 'ATD'] + 1 
                    line = f.readline()
                    s1 = line.split('\t') 
                    while (s1[-2] == 'I' or s1[-1][0] == 'I') and (s1[-2] != 'B' and s1[-1][0] != 'B'):
                        line = f.readline()
                        s1 = line.split()
                        if len(s1) > 1:
                            continue
                        else:
                            break
                    else: 
                        continue
                elif s1[-2] == 'B' and s1[-1][0] =='O' :         # CASE III 'BO'
                    line = f.readline()
                    s1 = line.split('\t') 
                    if len(s1) == 1:
                        line = f.readline()
                        continue
                    # BO follows *B
                    if  s1[-1][0] =='B' :
                        BIO_CM.loc['ATO', 'ATD'] = BIO_CM.loc['ATO', 'ATD'] + 1
                    # BO follows IO...IO...IO
                    elif s1[-2] == 'I' and s1[-1][0] =='O':
                        BIO_CM.loc['ATO', 'NATD'] = BIO_CM.loc['ATO', 'NATD'] + 1
                        line = f.readline()
                        s1 = line.split('\t')
                        if len(s1) == 1:
                            line = f.readline()
                            continue
                        while (s1[-2] != 'I' and s1[-1][0] == 'O') :
                            line = f.readline()
                            s1 = line.split()
                            if len(s1) > 1:
                                continue
                            else:
                                break
                        continue
                    # BO follows OO
                    elif s1[-2] == 'O' and s1[-1][0] =='O':        
                        BIO_CM.loc['ATO', 'NATD'] = BIO_CM.loc['ATO', 'NATD'] + 1
                    
            
                elif s1[-2] == 'O' and s1[-1][0] =='B' :                 # CASE IV 'OB' 
                    line = f.readline()
                    s1 = line.split('\t') 
                    if len(s1) == 1:
                        line = f.readline()
                        continue
                    # OB follows B*
                    if s1[-2] == 'B': # and s1[-1][0] =='I' :
                        BIO_CM.loc['ATO', 'ATD'] = BIO_CM.loc['ATO', 'ATD'] + 1
                    # OB follows OI...OI...OI
                    elif s1[-2] == 'O' and s1[-1][0] =='I':
                        BIO_CM.loc['NATO', 'ATD'] = BIO_CM.loc['NATO', 'ATD'] + 1
                        line = f.readline()
                        s1 = line.split('\t') 
                        if len(s1) == 1:
                            line = f.readline()
                            continue
                        while (s1[-2] == 'O' and s1[-1][0] == 'I') :
                            line = f.readline()
                            s1 = line.split()
                            if len(s1) > 1:
                                continue
                            else:
                                break
                        else:
                            continue                        
                    # OB follows OO
                    elif s1[-2] == 'O' and s1[-1][0] =='O':
                        BIO_CM.loc['NATO', 'ATD'] = BIO_CM.loc['NATO', 'ATD'] + 1  
            line = f.readline()
            
        f.close()
        
        
    
        Output['file_name'] = file_names[file_no]
        Output['Confusion_matrix'] = BIO_CM
        Precision = BIO_CM.loc['ATO', 'ATD']/(BIO_CM.loc['ATO', 'ATD']+BIO_CM.loc['ATO', 'NATD'])
        Output['Precision'] = Precision
        Recall = BIO_CM.loc['ATO', 'ATD']/(BIO_CM.loc['ATO', 'ATD']+BIO_CM.loc['NATO', 'ATD'])
        Output['Recall'] = Recall
        if Precision != 0 :
            F_measure = 2 * Precision * Recall / (Precision + Recall)  
        else:
            F_measure = 0
        Output['F_measure'] = F_measure
        ALL_OUTPUTS.append(Output)   
        
        
        
        run_details = '/////// FOR ' + file_names[file_no] + ', RUN 1\n'
        temp_out.write(run_details)
        confusion_matrix_str = str(BIO_CM.loc['ATO', 'ATD'])+ '\t ' + str(BIO_CM.loc['ATO', 'NATD']) + '\t' + str(BIO_CM.loc['NATO', 'ATD']) + '\t' + str(BIO_CM.loc['NATO', 'NATD'])+ '\n'        
        temp_out.write(confusion_matrix_str)
        Precision_str = 'Precision = ' + str(round(Precision,2))+'\n'
        temp_out.write(Precision_str)
        Recall_str = 'Recall = '+ str(round(Recall,2))+'\n'
        temp_out.write(Recall_str)
        F_measure_str = 'F-measure = ' +str(round(F_measure,2))+'\n'
        temp_out.write(F_measure_str)
        temp_out.write('\n')
        
        print(run_details)
        print(' CM ' ,BIO_CM)
        print(' P ',Precision_str)
        print('R ', Recall)
        print('FM', F_measure)
        
        
        
    

#temp_out.write(ALL_OUTPUTS)
temp_out.close()   
print('done')

['यह', 'DEM', 'none', 'O', 'O\n']
O
O
['हलका', 'NN', 'ा', 'B', 'O\n']
B
O
['\n']
['यह', 'DEM', 'none', 'O', 'O\n']
O
O
['ऑफ़लाइन', 'NN', 'ा', 'B', 'O\n']
B
O
['करता', 'VM', 'ता', 'O', 'O\n']
O
O
['है', 'VAUX', 'none', 'O', 'O\n']
O
O
['।', 'SYM', 'none', 'O', 'O\n']
O
O
['\n']
['यह', 'PRP', 'none', 'O', 'O\n']
O
O
['सीमित', 'JJ', 'ा', 'O', 'O\n']
O
O
['उपयोग', 'NN', 'ा', 'O', 'O\n']
O
O
['का', 'PSP', 'none', 'O', 'O\n']
O
O
['है।', 'SYM', 'ा', 'O', 'O\n']
O
O
['\n']
['इसमें', 'PRP', 'ें', 'O', 'O\n']
O
O
['अनुकूलन', 'NN', 'ा', 'B', 'O\n']
B
O
['अभाव', 'NN', 'ा', 'O', 'O\n']
O
O
['है', 'VM', 'none', 'O', 'O\n']
O
O
['।', 'SYM', 'none', 'O', 'O\n']
O
O
['\n']
['इस', 'DEM', 'none', 'O', 'O\n']
O
O
['एप्लीकेशन', 'NN', 'ा', 'O', 'O\n']
O
O
['की', 'PSP', 'none', 'O', 'O\n']
O
O
['इंस्टॉल', 'NN', 'ा', 'O', 'O\n']
O
O
['का', 'PSP', 'none', 'O', 'O\n']
O
O
['आकार', 'NN', 'ा', 'O', 'O\n']
O
O
['छोटा', 'JJ', 'ा', 'O', 'O\n']
O
O
['सा', 'RP', 'none', 'O', 'O\n']
O
O
['3', 'QC', 'none', 'O', 'O\n']

O
O
['इंटरनेट', 'NN', 'ा', 'B', 'O\n']
B
O
['\n']
['कुल', 'NN', 'ा', 'O', 'O\n']
O
O
['मिलाकर', 'VM', 'ाकर', 'O', 'O\n']
O
O
['यह', 'DEM', 'none', 'O', 'O\n']
O
O
['गेम', 'NN', 'ा', 'O', 'O\n']
O
O
['एक', 'QC', 'none', 'O', 'O\n']
O
O
['बेहतरीन', 'JJ', 'ा', 'O', 'O\n']
O
O
['आइडिया', 'NN', 'ा', 'O', 'O\n']
O
O
['है', 'VM', 'none', 'O', 'O\n']
O
O
['जो', 'PRP', 'none', 'O', 'O\n']
O
O
['घंटों', 'NN', 'ों', 'O', 'O\n']
O
O
['तक', 'PSP', 'none', 'O', 'O\n']
O
O
['यूजर्स', 'NNP', 'ा', 'B', 'O\n']
B
O
['अपने', 'PRP', 'ने', 'O', 'O\n']
O
O
['डिवाइस', 'NNP', 'ा', 'O', 'O\n']
O
O
['से', 'PSP', 'none', 'O', 'O\n']
O
O
['जोड़े', 'VM', 'े', 'O', 'O\n']
O
O
['रख', 'VM', 'none', 'O', 'O\n']
O
O
['सकता', 'VAUX', 'ता', 'O', 'O\n']
O
O
['है।', 'VAUX', 'ा', 'O', 'O\n']
O
O
['\n']
['चूंकि', 'CC', 'ि', 'O', 'O\n']
O
O
['प्लेयर्स', 'NN', 'ा', 'O', 'O\n']
O
O
['इस', 'DEM', 'none', 'O', 'O\n']
O
O
['प्रक्रिया', 'NN', 'ा', 'O', 'O\n']
O
O
['में', 'PSP', 'ा', 'O', 'O\n']
O
O
['अपनी', 'PRP', 'नी', 'O', 'O\n']


O
O
['जिसके', 'PRP', 'े', 'O', 'O\n']
O
O
['कारण', 'NN', 'ा', 'O', 'O\n']
O
O
['आपको', 'PRP', 'ो', 'O', 'O\n']
O
O
['एक्टिव', 'NNP', 'ा', 'O', 'O\n']
O
O
['जीपीआरएस', 'NNP', 'ा', 'O', 'O\n']
O
O
['कनेक्शन,', 'NNP', 'ा', 'O', 'O\n']
O
O
['इस', 'DEM', 'none', 'O', 'O\n']
O
O
['एप्प', 'NN', 'ा', 'O', 'O\n']
O
O
['को', 'PSP', 'none', 'O', 'O\n']
O
O
['प्रयोग', 'NN', 'ा', 'O', 'O\n']
O
O
['करने', 'VM', 'ने', 'O', 'O\n']
O
O
['के', 'PSP', 'none', 'O', 'O\n']
O
O
['लिए', 'PSP', 'ा', 'O', 'O\n']
O
O
['नहीं', 'NEG', 'ीं', 'O', 'O\n']
O
O
['लेना', 'VM', 'ना', 'O', 'O\n']
O
O
['पड़ता', 'VAUX', 'ता', 'O', 'O\n']
O
O
['है।', 'VAUX', 'ा', 'O', 'O\n']
O
O
['\n']
['यह', 'PRP', 'none', 'O', 'O\n']
O
O
['बहुत', 'QF', 'ा', 'O', 'O\n']
O
O
['ही', 'RP', 'none', 'O', 'O\n']
O
O
['उपयोगी', 'JJ', 'ोगी', 'O', 'O\n']
O
O
['एप्पलीकेशन', 'NN', 'ा', 'O', 'O\n']
O
O
['है', 'VM', 'none', 'O', 'O\n']
O
O
['।', 'SYM', 'none', 'O', 'O\n']
O
O
['\n']
['लेकिन', 'CC', 'ा', 'O', 'O\n']
O
O
['यह', 'PRP', 'none', 'O', 'O\n']

O
['साइज', 'NN', 'ा', 'B', 'O\n']
B
O
['1', 'QC', 'none', 'O', 'O\n']
O
O
['जीबी', 'NN', 'ी', 'O', 'O\n']
O
O
['है', 'VM', 'none', 'O', 'O\n']
O
O
['अगर', 'CC', 'ा', 'O', 'O\n']
O
O
['आप', 'PRP', 'none', 'O', 'O\n']
O
O
['वाईफाई', 'NNP', 'ाई', 'O', 'B\n']
O
B
['मदद', 'NN', 'ा', 'O', 'O\n']
O
O
['से', 'PSP', 'none', 'O', 'O\n']
O
O
['गेम', 'NN', 'ा', 'O', 'O\n']
O
O
['डाउनलोड', 'NN', 'ा', 'O', 'O\n']
O
O
['करते', 'VM', 'ते', 'O', 'O\n']
O
O
['हैं', 'VAUX', 'ा', 'O', 'O\n']
O
O
['तो', 'CC', 'none', 'O', 'O\n']
O
O
['ज्\u200dयादा', 'JJ', 'ा', 'O', 'O\n']
O
O
['बेहतर', 'JJ', 'ा', 'O', 'O\n']
O
O
['होगा।', 'VM', 'ा', 'O', 'O\n']
O
O
['\n']
['पेंटैक्स', 'NN', 'ा', 'O', 'O\n']
O
O
['के-500', 'PSP', 'ा', 'O', 'O\n']
O
O
['डीएसएलआर', 'NNP', 'ा', 'O', 'O\n']
O
O
[',शौकिया', 'NNP', 'ा', 'O', 'O\n']
O
O
['फोटोग्राफी', 'NN', 'ी', 'O', 'O\n']
O
O
['के', 'PSP', 'none', 'O', 'O\n']
O
O
['लिए', 'PSP', 'ा', 'O', 'O\n']
O
O
['अच्छे', 'JJ', 'े', 'O', 'O\n']
O
O
['फीचर्स', 'NN', 'ा', 'B', 'B\n']
B
B
['और',

O
['60पिक्सल/50पिक्सल', 'NNP', 'ा', 'O', 'O\n']
O
O
['इमेजेज', 'CC', 'ा', 'O', 'O\n']
O
O
['एसडी', 'NNP', 'ी', 'B', 'O\n']
B
O
['\n']
['पेनासोनिक', 'JJ', 'ा', 'O', 'O\n']
O
O
['कंपनी', 'NN', 'नी', 'O', 'O\n']
O
O
['अब', 'PRP', 'none', 'O', 'O\n']
O
O
['एक', 'QC', 'none', 'O', 'O\n']
O
O
['ऎसा', 'NN', 'ा', 'O', 'O\n']
O
O
['जबरदस्त', 'JJ', 'ा', 'O', 'O\n']
O
O
['कैमरा', 'NN', 'ा', 'O', 'O\n']
O
O
['लेकर', 'VM', 'कर', 'O', 'O\n']
O
O
['आई', 'VM', 'none', 'O', 'O\n']
O
O
['जिससें', 'NN', 'ें', 'O', 'O\n']
O
O
['आप', 'PRP', 'none', 'O', 'O\n']
O
O
['अपनी', 'PRP', 'नी', 'O', 'O\n']
O
O
['मर्जी', 'NN', 'ी', 'O', 'O\n']
O
O
['के', 'PSP', 'none', 'O', 'O\n']
O
O
['अनुसार', 'PSP', 'ा', 'O', 'O\n']
O
O
['शानदार', 'JJ', 'ा', 'O', 'O\n']
O
O
['क्वालिटी', 'NN', 'ी', 'O', 'O\n']
O
O
['वाली', 'PSP', 'ी', 'O', 'O\n']
O
O
['मूवी', 'NN', 'ी', 'B', 'O\n']
B
O
['कर', 'VM', 'none', 'O', 'O\n']
O
O
['सकते', 'VAUX', 'ते', 'O', 'O\n']
O
O
['हैं।', 'VAUX', 'ा', 'O', 'O\n']
O
O
['\n']
['पेनासोनिक', 'NNP', 'ा', 

...

O
['करते', 'VM', 'ते', 'O', 'O\n']
O
O
['बादल', 'NN', 'ा', 'O', 'O\n']
O
O
['और', 'CC', 'none', 'O', 'O\n']
O
O
['मन', 'NN', 'none', 'O', 'O\n']
O
O
['मोहती', 'VM', 'ती', 'O', 'O\n']
O
O
['हरियाली...', 'JJ', 'ा', 'B', 'O\n']
B
O
['के', 'PSP', 'none', 'O', 'O\n']
O
O
['कुछ', 'QF', 'ा', 'O', 'O\n']
O
O
['अदभुत', 'JJ', 'ा', 'O', 'O\n']
O
O
['दृश्यों', 'NN', 'ों', 'O', 'O\n']
O
O
['में', 'PSP', 'ा', 'O', 'O\n']
O
O
['से', 'PSP', 'none', 'O', 'O\n']
O
O
['ये', 'PRP', 'none', 'O', 'O\n']
O
O
['भी', 'RP', 'none', 'O', 'O\n']
O
O
['एक', 'QC', 'none', 'O', 'O\n']
O
O
['था।', 'NN', 'ा', 'O', 'O\n']
O
O
['\n']
['पहाड़', 'NN', 'ा', 'O', 'O\n']
O
O
['के', 'PSP', 'none', 'O', 'O\n']
O
O
['बीचों', 'NN', 'ों', 'O', 'O\n']
O
O
['बीच', 'NST', 'ा', 'O', 'O\n']
O
O
['पतले', 'JJ', 'े', 'O', 'O\n']
O
O
['झरने', 'VM', 'ने', 'O', 'O\n']
O
O
['की', 'PSP', 'none', 'O', 'O\n']
O
O
['सफेद', 'JJ', 'ा', 'O', 'O\n']
O
O
['लकीर,', 'JJ', 'ा', 'O', 'O\n']
O
O
['चट्टानों', 'NN', 'ों', 'O', 'O\n']
O
O
['के', 'PSP', 'none'

O
['भी', 'RP', 'none', 'O', 'O\n']
O
O
['दे', 'VM', 'none', 'O', 'O\n']
O
O
['रहा', 'VAUX', 'ा', 'O', 'O\n']
O
O
['था', 'VAUX', 'none', 'O', 'O\n']
O
O
['।', 'SYM', 'none', 'O', 'O\n']
O
O
['\n']
['चौड़ी', 'JJ', 'ी', 'O', 'O\n']
O
O
['पत्ती', 'NN', 'ती', 'O', 'O\n']
O
O
['वाले', 'PSP', 'े', 'O', 'O\n']
O
O
['पेड़ों', 'NN', 'ों', 'O', 'O\n']
O
O
['की', 'PSP', 'none', 'O', 'O\n']
O
O
['जगह', 'NN', 'ा', 'O', 'O\n']
O
O
['अब', 'PRP', 'none', 'O', 'O\n']
O
O
['नुकीली', 'JJ', 'ी', 'O', 'O\n']
O
O
['पत्ती', 'NN', 'ती', 'O', 'O\n']
O
O
['वाले', 'PSP', 'े', 'O', 'O\n']
O
O
['पेड़ो', 'NNP', 'ो', 'O', 'O\n']
O
O
['ने', 'PSP', 'none', 'O', 'O\n']
O
O
['ले', 'VM', 'none', 'O', 'O\n']
O
O
['ली', 'VAUX', 'none', 'O', 'O\n']
O
O
['थी', 'VAUX', 'none', 'O', 'O\n']
O
O
['।', 'SYM', 'none', 'O', 'O\n']
O
O
['\n']
['पर', 'CC', 'none', 'O', 'O\n']
O
O
['ये', 'PRP', 'none', 'O', 'O\n']
O
O
['क्या', 'QW', 'ा', 'O', 'O\n']
O
O
['थान्गू', 'NN', 'ू', 'O', 'O\n']
O
O
['पहुँचते', 'VM', 'ते', 'O', 'O\n']
O
O
['पहुँचत

O
['से', 'PSP', 'none', 'O', 'O\n']
O
O
['जाने', 'VM', 'ने', 'O', 'O\n']
O
O
['से', 'PSP', 'none', 'O', 'O\n']
O
O
['समय', 'NN', 'ा', 'O', 'O\n']
O
O
['कम', 'QF', 'none', 'O', 'O\n']
O
O
['लगेगा', 'VM', 'ेगा', 'O', 'O\n']
O
O
['और', 'CC', 'none', 'O', 'O\n']
O
O
['ऊँचाई', 'NN', 'ाई', 'O', 'O\n']
O
O
['से', 'PSP', 'none', 'O', 'O\n']
O
O
['नयनाभिराम', 'JJ', 'ा', 'O', 'O\n']
O
O
['दृश्य', 'NN', 'ा', 'O', 'O\n']
O
O
['दिखेंगे', 'VM', 'ेंगे', 'O', 'O\n']
O
O
['सो', 'VAUX', 'none', 'O', 'O\n']
O
O
['अलग।', 'VAUX', 'ा', 'O', 'O\n']
O
O
['\n']
['माउंट', 'NNP', 'ा', 'O', 'O\n']
O
O
['एवरेस्ट', 'NNP', 'ा', 'O', 'O\n']
O
O
['को', 'PSP', 'none', 'O', 'O\n']
O
O
['देख', 'VM', 'ा', 'O', 'O\n']
O
O
['पाने', 'VAUX', 'ने', 'O', 'O\n']
O
O
['की', 'PSP', 'none', 'O', 'O\n']
O
O
['ललक', 'NN', 'ा', 'O', 'O\n']
O
O
['भी', 'RP', 'none', 'O', 'O\n']
O
O
['अंदर', 'NST', 'ा', 'O', 'O\n']
O
O
['ही', 'RP', 'none', 'O', 'O\n']
O
O
['अंदर', 'NST', 'ा', 'O', 'O\n']
O
O
['स्कूली', 'JJ', 'ी', 'O', 'O\n']
O
O
['मन', '

O
['कम', 'QF', 'none', 'O', 'O\n']
O
O
['डेंस', 'JJ', 'ा', 'O', 'O\n']
O
O
['है,', 'NN', 'ा', 'O', 'O\n']
O
O
['लेकिन', 'CC', 'ा', 'O', 'O\n']
O
O
['जितना', 'QF', 'ना', 'O', 'O\n']
O
O
['एक', 'QC', 'none', 'O', 'O\n']
O
O
['डेस्कटॉप', 'NN', 'ा', 'O', 'O\n']
O
O
['को', 'PSP', 'none', 'O', 'O\n']
O
O
['शॉर्प', 'JJ', 'ा', 'O', 'O\n']
O
O
['होना', 'VM', 'ना', 'O', 'O\n']
O
O
['चाहिए,', 'NN', 'ा', 'O', 'O\n']
O
O
['उससे', 'PRP', 'े', 'O', 'O\n']
O
O
['ज्यादा', 'QF', 'ा', 'O', 'O\n']
O
O
['शार्प', 'NN', 'ा', 'O', 'O\n']
O
O
['है।', 'VM', 'ा', 'O', 'O\n']
O
O
['\n']
['इसमें', 'PRP', 'ें', 'O', 'O\n']
O
O
['4', 'QC', 'none', 'O', 'O\n']
O
O
['यूएसबी', 'NNP', 'ी', 'B', 'O\n']
B
O
['पोर्ट,', 'NNP', 'ा', 'I', 'I\n']
I
I
['2', 'QC', 'none', 'B', 'O\n']
B
O
['पोर्ट,', 'NNP', 'ा', 'I', 'O\n']
I
O
['3.5एमएम', 'NNP', 'ा', 'B', 'O\n']
B
O
['सॉकेट', 'NNP', 'ा', 'I', 'O\n']
I
O
['आदि', 'RP', 'ि', 'O', 'O\n']
O
O
['है,', 'VM', 'ा', 'O', 'O\n']
O
O
['लेकिन', 'CC', 'ा', 'O', 'O\n']
O
O
['इसकी', 'PRP', 'ी', 

done


In [8]:
def trn_tst_count(sizes,training_percent):
    num_training = []
    num_testing = []
    offset = []

    for i in range(0,len(sizes)):
        num_train = round(sizes[i]* training_percent /100)
        num_training.append(num_train)
        num_test = sizes[i]-num_training[i]
        num_testing.append(num_test)
        if i==0:
            offset.append(0)
        else:
            offset.append(offset[i-1]+sizes[i-1])
             
    return(num_training,num_testing,offset)

sizes = [229,150,43,84,348,1141,890,330,47,1244,776,135]
num_training,num_testing,offset = trn_tst_count(sizes,80)
print(num_training)
print(num_testing)
print(sizes)
print(offset)

[183, 120, 34, 67, 278, 913, 712, 264, 38, 995, 621, 108]
[46, 30, 9, 17, 70, 228, 178, 66, 9, 249, 155, 27]
[229, 150, 43, 84, 348, 1141, 890, 330, 47, 1244, 776, 135]
[0, 229, 379, 422, 506, 854, 1995, 2885, 3215, 3262, 4506, 5282]


In [9]:
import numpy
# x is your dataset
x = numpy.random.rand(100)
sizes = [229,150,43,84,348,1141,890,330,47,1244,776,135]
training_percent = 80

## Function call to
# to 
#
num_training,num_testing,offset = trn_tst_count(sizes,training_percent)
print(offset)


training =[]
testing = []
num_training = []
for i in range(0,len(sizes)):
    num_train = round(sizes[i]* training_percent /100)
    num_training.append(num_train)
    indices = numpy.random.permutation(sizes[i])
    training_idx, testing_idx = indices[:num_training[i]], indices[num_training[i]:]
    training.append(training_idx+offset[i])
    testing.append(testing_idx+offset[i])
    print("For domain ", i , "TRAIN\n", training_idx+offset[i])
    print("For domain ", i , "TEST\n", testing_idx+offset[i])
    #print(x)
    
    print(len(training_idx))
    print(" ",len(testing_idx),"\n")

print(training)
print(testing)

[0, 229, 379, 422, 506, 854, 1995, 2885, 3215, 3262, 4506, 5282]
For domain  0 TRAIN
 [ 65  49 180  33 173 206 217 227  58  32   7 187 148  68  94 151 124  99
 159  93 208  56 142  51  23  20 177 214 195  89 171 191  72 156  12 119
 183  38 102 120  78 158  88  14 200  16 223 222  85  48  69 181  31  92
  29 153 146  27  18 178 184  26 128 169 199 108   4 188  61  55  42 133
  95   2  35  73 135 210  59 186  54 189 127 116  53 190 149  60  11 226
 193  24 176  17   5 114 175 228 216 109  81 111  71 221 213 155 152 132
  62 143 125 147 160 117  37 101 201 121  80 179 224 174  90  96 211 137
  21 110  22  86 215  91  25 170  76 138 130  10  83  45 118  50 220 103
  63  75  64  98  97   9 204 168 218 123 163  46 107 139  40 112 157 165
 209 212 144 192 194  79  47 172  28 225 202 162  87  66 122  39   8  77
 164 126  43]
For domain  0 TEST
 [  0  15 134  41 182 154 106 219 131 136 141 196 150 113  74  44   3 129
 205  82  13 166 115   1 207 203  57 100 161 104  36  67 198  84 167 145
   6

In [28]:
file_names = ['Mobile_apps','Camera']#,'Headphones','Home_appliances','Laptop','Mobile','Movies','Smart_watch','Speakers','Tablets','Travels','Televisions']
sizes = [229,150,43,84,348,1141,890,330,47,1244,776,135 ]
file_no = 0

print (temp_train)
print (temp_test)
all_trnfp = open("Training_All", "w", encoding="utf8")
all_tstfp = open("Testing_All", "w", encoding="utf8")

for j in range(0, len(file_names)):
    trnstr = "Training_" + file_names[j] 
    tststr = "Testing_" + file_names[j] 
    trnfp = open(trnstr, "w", encoding = "utf8")
    tstfp = open(tststr, "w", encoding= "utf8")

    
    # For indexes randomly choosen from training and testing 
    
    temp_train = training[j]
    temp_test = testing[j]
    
    
    for i in range(offset[j], offset[j] + sizes[j]):
        
        # Get data of each every word in the sentence numbered i
        records = get_train_test_data(i)
        num_records = len(records)


        if i in temp_train:
            for k in range(0, num_records):
                words = records[k]['word'] + "\t" + records[k]['pos'] + "\t" + records[k]['BIO'] + "\t" + records[k]['w_suff'] + "\n"
                trnfp.write(words)
                all_trnfp.write(words)
            trnfp.write("\n")
            all_trnfp.write("\n")

            #print ("Put in training ")
        else:
            for k in range(0, num_records):
                words = records[k]['word'] + "\t" + records[k]['pos'] + "\t" + records[k]['BIO'] + "\t" + records[k]['w_suff'] + "\n"
                tstfp.write(words)
                all_tstfp.write(words)
            tstfp.write("\n")
            all_tstfp.write("\n")
            
            #print ("Put in test")

    trnfp.close()
    tstfp.close()
print ("Done")

[ 65  49 180  33 173 206 217 227  58  32   7 187 148  68  94 151 124  99
 159  93 208  56 142  51  23  20 177 214 195  89 171 191  72 156  12 119
 183  38 102 120  78 158  88  14 200  16 223 222  85  48  69 181  31  92
  29 153 146  27  18 178 184  26 128 169 199 108   4 188  61  55  42 133
  95   2  35  73 135 210  59 186  54 189 127 116  53 190 149  60  11 226
 193  24 176  17   5 114 175 228 216 109  81 111  71 221 213 155 152 132
  62 143 125 147 160 117  37 101 201 121  80 179 224 174  90  96 211 137
  21 110  22  86 215  91  25 170  76 138 130  10  83  45 118  50 220 103
  63  75  64  98  97   9 204 168 218 123 163  46 107 139  40 112 157 165
 209 212 144 192 194  79  47 172  28 225 202 162  87  66 122  39   8  77
 164 126  43]
[  0  15 134  41 182 154 106 219 131 136 141 196 150 113  74  44   3 129
 205  82  13 166 115   1 207 203  57 100 161 104  36  67 198  84 167 145
   6  70 140  34  19 105  30 185  52 197]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [ ]:
import subprocess
cmd = ['shallow_parser_hin', 'IN_OUT/INPFILE', 'IN_OUT/OUTFILE']
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE)
    try:dde
        out, err = p.communicate()
    except p.CalledProcessError as e:
        print ("Exception:", row['text'])
        print (e.output)
        continue